In [ ]:
using PyCall
using PyPlot
using DifferentialEquations


Bad key text.latex.unicode in file /home/arurz/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.0/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.frameon in file /home/arurz/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 423 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.0/matplotlibrc.template
or from the matplotlib source distribution

Bad key pgf.debug in file /home/arurz/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 444 ('pgf.debug           : False')
You probably need to get an updated matplotlibrc file from
https://github.com/

In [ ]:
spy = pyimport("scipy.interpolate")
np = pyimport("numpy")
mp = pyimport("mpmath")

In [ ]:
Δ = 1/2
ω = 2.5

;

In [ ]:
function k(t)
    return Δ + cos(ω*t)
end

function k1(t,κ)
    return 2*κ^2/cosh(κ*t)^2
end

In [ ]:
function osc!(du,u,p,t)
 du[1] = u[2]
 du[2] = (1/u[1]^3)-k(t)*u[1]
 du[3] = (1/u[1]^2)
end

In [ ]:
u0 = [1.0;0.0;0.0]
tspan = (0.0,20)
prob = ODEProblem(osc!,u0,tspan)
sol = solve(prob,Vern7(),dt=0.00000000001)
;

In [ ]:
t = np.linspace(np.min(sol.t), np.max(sol.t), 200)  

spl1 = spy.make_interp_spline(sol.t, sol'[:,1], k=3)
spl2 = spy.make_interp_spline(sol.t, sol'[:,2], k=3)
spl3 = spy.make_interp_spline(sol.t, sol'[:,3], k=3)

ps1 = spl1(t)
ps2 = spl2(t)
ps3 = spl3(t)

;

In [ ]:
rc("text", usetex=true)

In [ ]:
fig,(ax1,ax2,ax3)=plt.subplots(3,1,figsize=(5,5),sharex=true)
plt.subplots_adjust(hspace = 0.1)

ax1.plot(t,ps1,"b-", linewidth=2,label=L"\omega = 2.5\qquad \Delta=1/2")
ax2.plot(t,ps2,"b-", linewidth=2)
ax3.plot(t,ps3,"b-", linewidth=2)

ax1.set_aspect("auto")
ax2.set_aspect("auto")
ax3.set_aspect("auto")

ax1.tick_params(direction="out",length=5,width=2,labelsize=14)
ax2.tick_params(direction="out",length=5,width=2,labelsize=14)
ax3.tick_params(direction="out",length=5,width=2,labelsize=14)

ax1.grid(linewidth=1)
ax2.grid(linewidth=1)
ax3.grid(linewidth=1)

ax1.set_ylabel(L"\rho(t)",fontsize=16)
ax2.set_ylabel(L"\dot{\rho}(t)",fontsize=16)
ax3.set_ylabel(L"\omega_{\rho}(t)",fontsize=16)

ax3.set_xlabel(L"t\ [s]",fontsize=16,labelpad=0)
xlim(0,20)

ax1.legend(bbox_to_anchor=(0.3,0.3,0.5,1),loc=9,ncol=1, mode="expand", borderaxespad=0,fontsize=11)

show()

savefig("osckvn.png", transparent = "true", dpi=300)

In [ ]:
function β1(t,ρ0)
    return convert(Float64,(ρ0/sol'[t,1])*mp.cos(sol'[t,3])+ρ0*sol'[t,2]*mp.sin(sol'[t,3]))
end

function β2(t,ρ0)
    return convert(Float64,ρ0*sol'[t,1]*mp.sin(sol'[t,3]))
end

function β3(t,ρ0,dρ0)
    return convert(Float64,(1/(ρ0*sol'[t,1])+dρ0*sol'[t,2])*mp.sin(sol'[t,3])+((dρ0/sol'[t,1])-(sol'[t,2]/ρ0))*mp.cos(sol'[t,3]))
end

function β4(t,ρ0,dρ0)
    return convert(Float64,(sol'[t,1]/ρ0)mp.cos(sol'[t,3])-dρ0*sol'[t,1]*mp.cos(sol'[t,3]))
end

;

In [ ]:
function qc(q,p,t)
    return q*β1(t,1)-p*β2(t,1)
end
    
function pc(q,p,t)
    return q*β3(t,1,0)+p*β4(t,1,0)
end

;

In [ ]:
qv = zeros(Float64,length(sol.t))
for i in 1:length(sol.t)
    qv[i] = qc(2,2,i)
end
pv = zeros(Float64,length(sol.t))
for i in 1:length(sol.t)
    pv[i] = pc(2,2,i)
end

In [ ]:
t1 = np.linspace(np.min(sol.t), np.max(sol.t), 200)  

sp1 = spy.make_interp_spline(sol.t, qv, k=3)
sp2 = spy.make_interp_spline(sol.t, pv, k=3)

ps1 = sp1(t)
ps2 = sp2(t)

;

In [ ]:
figps = figure(figsize=(5,5))
axps = gca(projection="3d")
plot3D(ps1,ps2,t1,"b-", linewidth=2)
axps.view_init(30,-30)
xlabel(L"q",fontsize=16)
ylabel(L"p",fontsize=16)
zlabel(L"t\ [s]",fontsize=16)
axps.tick_params(axis="both",direction="out",length=10,width=3,grid_linewidth=3,labelsize=14)
axps.set_zticks(range(0,stop=20,step=5))
axps.set_zticklabels([L"0",L"5",L"10",L"15",L"20"])
show()
tight_layout()

savefig("oscckvn.png", transparent = "true", dpi=300)